# Manejo de Datos en PyTorch

## Librerías

In [7]:
import csv
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import tempfile

from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset

## La clase Dataset

La clase abstracta [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) es la clase base para construir un dataset de PyTorch. Cualquier dataset personalizado debe heredar de dicha clase e implementar los siguientes métodos:

- `__len__`: Para que `len(dataset)` devuelva el tamaño del conjunto de datos.
- `__getitem__`: Para soportar indexado de manera que `dataset[i]` devuelva el elemento `i`. Es común que en ciertos casos se utilice este método para levantar el dato real (e.g. una imagen) mientras que lo que se guarde en el dataset sea sólo una referencia a dicho dato (e.g. un path a la imagen). De esta manera se evita cargar muchas imágenes en memoria, haciendo que sea menos demandante a nivel RAM.

In [8]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
class IMDBReviewsDataset(Dataset):
    def __init__(self, path, transform=None):
        self.dataset = pd.read_csv(path)
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.tolist()  # Deal with list of items instead of tensor
        
        item = {
            "data": self.dataset.iloc[item]["review"],
            "target": self.dataset.iloc[item]["sentiment"]
        }

        if self.transform:
            item = self.transform(item)
        
        return item

dataset = IMDBReviewsDataset("/content/drive/MyDrive/DIPLO_DATOS/imdb_reviews.csv.gz")
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

Dataset loaded with 50000 elements
Sample element:
{'data': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><b

## Transformaciones

El ejemplo anterior nos muestra el uso básico, pero claramente no podemos pasarle eso a una red neuronal, no puede manejar texto. Es para eso que tenemos que hacer algún tipo de transformación sobre los atributos (en este caso el único atributo es el texto). 

### Normalización

En particular, como vemos en el caso anterior, el texto no está normalizado, parte de las transformaciones pueden incluir realizar algún tipo de normalización. Para eso hagamos uso de [`gensim`](https://radimrehurek.com/gensim/index.html), en particular utilizaremos el módulo [`preprocessing`](https://radimrehurek.com/gensim/parsing/preprocessing.html#module-gensim.parsing.preprocessing) que se encargará de hacer varias normalizaciones por defecto

In [10]:
class TextPreprocess:
    def __init__(self, filters=None):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _encode_target(self, target):
        return 1 if target == "positive" else 0

    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self._preprocess_string(item["data"])
        else:
            data = [self._preprocess_string(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self._encode_target(item["target"])
        else:
            target = [self._encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

preprocess = TextPreprocess()
print(preprocess(dataset[0]))

{'data': ['reviewers', 'mentioned', 'watching', 'episode', 'hooked', 'right', 'exactly', 'happened', 'thing', 'struck', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'right', 'word', 'trust', 'faint', 'hearted', 'timid', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence', 'hardcore', 'classic', 'use', 'word', 'called', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focuses', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda', 'city', 'home', 'aryans', 'muslims', 'gangstas', 'latinos', 'christians', 'italians', 'irish', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'far', 'away', 'main', 'appeal', 'fact', 'goes', 'shows', 'wouldn', 'dare', 'forget', 'pretty', 'pictures', 'painted', 'mainstream', 'audiences', 'forget', 'charm', 'forget', 'romance', 'mess', 'episode', 'saw', 'struck', 'nasty', 'surreal', 'couldn', 'ready',

In [11]:
dataset[1]

{'data': 'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terrib

In [12]:
preprocess(dataset[1])

{'data': ['wonderful',
  'little',
  'production',
  'filming',
  'technique',
  'unassuming',
  'old',
  'time',
  'bbc',
  'fashion',
  'gives',
  'comforting',
  'discomforting',
  'sense',
  'realism',
  'entire',
  'piece',
  'actors',
  'extremely',
  'chosen',
  'michael',
  'sheen',
  'got',
  'polari',
  'voices',
  'pat',
  'truly',
  'seamless',
  'editing',
  'guided',
  'references',
  'williams',
  'diary',
  'entries',
  'worth',
  'watching',
  'terrificly',
  'written',
  'performed',
  'piece',
  'masterful',
  'production',
  'great',
  'master',
  'comedy',
  'life',
  'realism',
  'comes',
  'home',
  'little',
  'things',
  'fantasy',
  'guard',
  'use',
  'traditional',
  'dream',
  'techniques',
  'remains',
  'solid',
  'disappears',
  'plays',
  'knowledge',
  'senses',
  'particularly',
  'scenes',
  'concerning',
  'orton',
  'halliwell',
  'sets',
  'particularly',
  'flat',
  'halliwell',
  'murals',
  'decorating',
  'surface',
  'terribly'],
 'target': 1

### Conversión a vectores

Podemos continuar convertiendo el texto en una representación por vectores. Si bien hay muchas posibilidades (siendo la bolsa de palabras una de las más utilizadas), en general para Deep Learning se prefieren representaciones utilizando vectores contínuos, obtenidos por algún método del estilo de Word2Vec, Glove o FastText. Para este caso utilizaremos las representaciones de Glove de dimensión 50 que se dejaron para descargar en el [notebook 0](./0_set_up.ipynb).

In [5]:
!pip install gensim --upgrade 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [14]:
class VectorizeText:
    def __init__(self, glove_vectors_path):
        self.glove_model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/DIPLO_DATOS/glove.6B.50d.txt.gz",
                                                             binary=False, no_header=True)
        self.unkown_vector = np.random.randn(self.glove_model.vector_size)  # Random vector for unknown words
    
    def _get_vector(self, word):
        if word in self.glove_model:
            return self.glove_model[word]
        else:
            return self.unkown_vector
    
    def _get_vectors(self, sentence):
        return np.vstack([self._get_vector(word) for word in sentence])
    
    def __call__(self, item):
        review = []
        if isinstance(item["data"][0], str):
            review = self._get_vectors(item["data"])
        else:
            review = [self._get_vectors(d) for d in item["data"]]

        return {
            "data": review,
            "target": item["target"]
        }

vectorizer = VectorizeText("/content/drive/MyDrive/DIPLO_DATOS/glove.6B.50d.txt.gz")
vectorizer(preprocess(dataset[0]))

{'data': array([[-0.18105   , -0.79229999, -0.097616  , ...,  1.42859995,
         -0.032471  ,  0.47235999],
        [ 0.69395   ,  0.69261003, -0.21608   , ...,  0.2247    ,
         -0.23197   ,  0.0062523 ],
        [-0.0049087 ,  0.12611   ,  0.14056   , ..., -0.58464003,
         -0.31830999,  0.31564   ],
        ...,
        [ 0.25435999, -0.44304001, -0.12524   , ...,  0.73352998,
          0.026198  ,  0.30408001],
        [-0.058468  ,  0.019087  ,  0.089056  , ..., -0.28176001,
          0.045137  , -0.18802001],
        [ 0.14443   ,  0.39103001, -0.93454999, ..., -0.71325999,
         -0.54575998,  0.13952   ]]), 'target': 1}

### Combinación de vectores

Si bien ahora estamos con una versión de los atributos que podría pasar por una red neuronal, hay un problema, las distintas reviews tienen largo distinto y como el algoritmo se entrena en lotes (*mini-batches*) estas requieren tener todas el mismo largo. Hay varias maneras de lidiar con esto, cada una con sus ventajas y desventajas. Dado que por ahora solo vimos perceptrón multicapa, que espera algo de tamaño fijo, una opción sencilla puede ser la de simplemente promediar los vectores de palabras.

In [15]:
class WordVectorsAverage:
    def __call__(self, item):
        if item["data"][0].ndim == 2:
            data = np.vstack([np.mean(d, axis=0) for d in item["data"]])
        else:
            data = np.mean(item["data"], axis=0)
        
        return {
            "data": data,
            "target": item["target"]
        }

vector_average = WordVectorsAverage()
vector_average(vectorizer(preprocess(dataset[0])))

{'data': array([ 0.11392873,  0.01126639, -0.12531332, -0.26262102,  0.2671047 ,
         0.0603304 , -0.05672802, -0.03585446, -0.15122386,  0.18890972,
        -0.21092786, -0.0878156 , -0.14540398, -0.01078587,  0.26998903,
         0.01295308,  0.02724838, -0.00719027, -0.131633  , -0.2576076 ,
        -0.05534939,  0.31699218,  0.13338708,  0.18708014,  0.10132058,
        -1.08588475, -0.35363793,  0.18712463,  0.44626267, -0.17460003,
         2.028427  ,  0.14113671, -0.05313083, -0.41163912, -0.07523687,
         0.10386484, -0.05572449, -0.15579046, -0.20278573, -0.23940431,
        -0.13492219,  0.06878732,  0.06135851,  0.26125413,  0.14199037,
         0.00565598, -0.00214863, -0.151335  , -0.02123343, -0.06164718]),
 'target': 1}

### Conversión de vectores a tensores

En el paso final, debemos convertir nuestros datos de arrays de `numpy` a tensores de PyTorch.

In [16]:
class ToTensor:
    def __call__(self, item):
        """
        This espects a single array.
        """
        return {
            "data": torch.from_numpy(item["data"]),
            "target": torch.tensor(item["target"])
        }

to_tensor = ToTensor()
to_tensor(vector_average(vectorizer(preprocess(dataset[0]))))

{'data': tensor([ 0.1139,  0.0113, -0.1253, -0.2626,  0.2671,  0.0603, -0.0567, -0.0359,
         -0.1512,  0.1889, -0.2109, -0.0878, -0.1454, -0.0108,  0.2700,  0.0130,
          0.0272, -0.0072, -0.1316, -0.2576, -0.0553,  0.3170,  0.1334,  0.1871,
          0.1013, -1.0859, -0.3536,  0.1871,  0.4463, -0.1746,  2.0284,  0.1411,
         -0.0531, -0.4116, -0.0752,  0.1039, -0.0557, -0.1558, -0.2028, -0.2394,
         -0.1349,  0.0688,  0.0614,  0.2613,  0.1420,  0.0057, -0.0021, -0.1513,
         -0.0212, -0.0616], dtype=torch.float64), 'target': tensor(1)}

### Componiendo las transformaciones

Para evitar tener que llamar a todas las funciones de transformación que querramos aplicar, para ello hacemos uso del parámetro `transform` que definimos en nuestro `Dataset` y un poco de ayuda de `functools`.

In [17]:
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions, lambda x: x)

dataset = IMDBReviewsDataset("/content/drive/MyDrive/DIPLO_DATOS/imdb_reviews.csv.gz",
                             transform=compose(preprocess, vectorizer, vector_average, to_tensor))
print(f"Dataset loaded with {len(dataset)} elements")
print(f"Sample element:\n{dataset[0]}")

Dataset loaded with 50000 elements
Sample element:
{'data': tensor([ 0.1139,  0.0113, -0.1253, -0.2626,  0.2671,  0.0603, -0.0567, -0.0359,
        -0.1512,  0.1889, -0.2109, -0.0878, -0.1454, -0.0108,  0.2700,  0.0130,
         0.0272, -0.0072, -0.1316, -0.2576, -0.0553,  0.3170,  0.1334,  0.1871,
         0.1013, -1.0859, -0.3536,  0.1871,  0.4463, -0.1746,  2.0284,  0.1411,
        -0.0531, -0.4116, -0.0752,  0.1039, -0.0557, -0.1558, -0.2028, -0.2394,
        -0.1349,  0.0688,  0.0614,  0.2613,  0.1420,  0.0057, -0.0021, -0.1513,
        -0.0212, -0.0616], dtype=torch.float64), 'target': tensor(1)}


### Iterando el dataset

Ya tenemos nuestro conjunto de datos con sus respectivas transformaciones. ¿Para qué nos sirve esto? Una opción es simplemente iterar en el conjunto de datos de a un elemento. Esto es sencillo, simplemente se hace a través de un `for`.

In [18]:
for idx, sample in enumerate(dataset):
    print(sample["data"])
    print(sample["target"])
    print("=" * 50)
    
    if idx == 2:
        break

tensor([ 0.1139,  0.0113, -0.1253, -0.2626,  0.2671,  0.0603, -0.0567, -0.0359,
        -0.1512,  0.1889, -0.2109, -0.0878, -0.1454, -0.0108,  0.2700,  0.0130,
         0.0272, -0.0072, -0.1316, -0.2576, -0.0553,  0.3170,  0.1334,  0.1871,
         0.1013, -1.0859, -0.3536,  0.1871,  0.4463, -0.1746,  2.0284,  0.1411,
        -0.0531, -0.4116, -0.0752,  0.1039, -0.0557, -0.1558, -0.2028, -0.2394,
        -0.1349,  0.0688,  0.0614,  0.2613,  0.1420,  0.0057, -0.0021, -0.1513,
        -0.0212, -0.0616], dtype=torch.float64)
tensor(1)
tensor([ 0.0973,  0.1677, -0.4141, -0.1020,  0.3493,  0.1831, -0.2603, -0.1201,
        -0.1452,  0.3607, -0.0804,  0.2755, -0.1421,  0.1363,  0.2668, -0.0615,
         0.1036,  0.1077, -0.1338, -0.3833,  0.0918,  0.3751,  0.0076,  0.0359,
         0.3707, -0.7851, -0.5731,  0.1247,  0.4316, -0.1730,  2.0905, -0.1258,
         0.0899, -0.4389,  0.0269,  0.3091, -0.0243,  0.2711, -0.2218, -0.3285,
         0.1263,  0.1229, -0.1221,  0.0448, -0.0204,  0.0761, 

## La clase Dataloader

El problema con iterar de a un elemento es que estamos limitados al querer entrenar un modelo. Por empezar, los modelos de Deep Learning suelen ser más eficientes si se entrenan utilizando algún tipo de entrenamiento por *mini-batches*. Además, hay otras cosas como mezclar los elementos (*shuffling*) o cargar datos en paralelo vía distintos *multiprocess workers*. La clase [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) precisamente se encarga de hacer eso por nosotros:

In [19]:
dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2
)

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, 
          sample_batched['data'].size(),
          sample_batched['target'].size())

    if i_batch == 2:
        print(sample_batched["data"])
        print(sample_batched["target"])
        break

0 torch.Size([4, 50]) torch.Size([4])
1 torch.Size([4, 50]) torch.Size([4])
2 torch.Size([4, 50]) torch.Size([4])
tensor([[ 7.5445e-02,  1.4737e-01, -2.6046e-02, -5.4799e-02,  1.5319e-01,
          2.0545e-01, -4.8994e-01,  2.1197e-02, -3.3329e-02,  2.7525e-01,
         -3.5378e-03,  1.7694e-01, -2.3254e-01,  1.3706e-01,  6.1033e-01,
          5.0111e-03,  1.4651e-01,  2.1230e-01, -3.3609e-01, -2.4830e-01,
          2.6905e-01,  3.7206e-01,  1.9186e-01,  2.4245e-01,  2.3919e-01,
         -1.2266e+00, -4.1710e-01, -1.9385e-02,  1.9557e-01, -2.2143e-01,
          2.3456e+00, -1.6921e-03, -9.2842e-02, -3.9659e-01, -1.5157e-02,
          1.6227e-01,  5.2807e-02, -1.0371e-01, -6.9218e-02, -3.8557e-01,
         -7.7057e-02,  1.6551e-01, -8.7824e-02, -9.6049e-02, -6.6318e-02,
          5.4025e-03, -1.3016e-01, -2.6332e-01, -6.6194e-02,  3.9274e-02],
        [ 1.9271e-01,  1.0231e-01, -1.0935e-01, -2.0108e-01,  4.7889e-01,
          1.8703e-01, -2.8711e-01,  1.3726e-01, -1.5207e-01,  1.8212e-0

## La clase IterableDataset

El método preferido para trabajar con conjuntos de datos en PyTorch es `torch.utils.data.Dataset`. En general, hacer uso inteligente del método `__getitem__`, e.g. usándolo para cargar imágenes a medida que sean necesitadas y no al instanciar el dataset, es la mejor manera de trabajar con un conjunto de datos. En particular, de esta forma es mucho más fácil hacer *shuffling* de los datos y demás. No obstante, no siempre esto es posible, muchas veces el conjunto de datos es demasiado grande para levantarlo en memoria (aunque sólo levantemos referencias). Para esos casos, PyTorch ofrece la clase [`torch.utils.data.IterableDataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset), en este caso el único método que es requerido implementar es `__iter__`.

In [ ]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None):
        self.dataset_path = path
        self.transform = transform

    def __iter__(self):
        with gzip.open(self.dataset_path, "rt") as fh:
            csv_reader = csv.reader(fh)
            _ = next(csv_reader)  # Remove the header
            for (_, _, title, category) in csv_reader:
                item = {
                    "data": title,
                    "target": category
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item

dataset = MeLiChallengeDataset("./data/meli-challenge-2019/spanish.train.csv.gz")
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0)
dataiter = iter(dataloader)
print(f"Sample batch:\n{dataiter.next()}")